In [1]:
from skopt.sampler import Lhs
from skopt.space import Space
import os 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from distutils.dir_util import copy_tree


In [2]:
# define constant
CRIPPLE_WALL_WEIGHT = 14
CRIPPLE_WALL_TRIB_LENGTH = [17.5, 15, 17.5, 20, 0, 20, 17.5, 15, 17.5]

WEIGHT = {1: np.array([[2.239, 3.617, 2.632, 3.035, 4.863, 3.697, 2.239, 3.62, 2.239], 
[2.810, 4.364, 3.204, 3.812, 5.623, 4.474, 2.810, 4.367, 2.810]]),
2: np.array([[2.239, 3.617, 2.632, 3.035, 4.863, 3.697, 2.239, 3.62, 2.239], 
[3.769, 5.461, 4.163, 4.636, 5.631, 5.298, 3.769, 5.464, 3.769],
[3.002, 4.530, 3.002, 4.039, 5.128, 4.039, 3.002, 4.530, 3.002]])}

CRIPPLE_WALL_MAT = 23
CRIPPLE_RETRO_MAT = 31
SUPER_INT_MAT = 15
SUPER_EXT_MAT = 5

Target_folder = r'C:\Users\User\Desktop\ExperimentDesign\OS Models\BuildingInfo'

seismic_weight_multiplier = [1, 0.5, 1.5]
damping_ratio = [0.03, 0.01, 0.05]
cripple_wall_height = [4, 2, 6]
force = [1, 0.5, 1.5]
disp = [1, 0.5, 1.5]

i = 1

design_mat = pd.DataFrame(index=range(243), columns=['seismic_weight_multiplier','damping_ratio', 'cripple_wall_height', 'force','disp'])
for s in seismic_weight_multiplier:
    for d in damping_ratio:
        for c in cripple_wall_height:
            for f in force:
                for dis in disp:

                    design_mat.iloc[i-1,:] = [s, d, c, f, dis]

                    copy_tree(r'C:\Users\User\Desktop\\ExperimentDesign\OS Models\Baseline Building Info\1L-S2-G2-2C-S2R2-SDSx', os.path.join(Target_folder, '%s-%i'%('1L-S2-G2-2C-S2R2-SDSx', i)))

                    current_model_dir = os.path.join(Target_folder, '%s-%i'%('1L-S2-G2-2C-S2R2-SDSx', i))
                    current_model_id = '%s-%i'%('1L-S2-G2-2C-S2R2-SDSx', i)

                    # modify damping 
                    damping = d
                    os.chdir(os.path.join(current_model_dir, 'AnalysisParameters/DynamicAnalysis'))
                    np.savetxt('dampingRatio.txt', np.array([round(damping,4)]), fmt = '%.3f')

                    os.chdir(os.path.join(current_model_dir, 'DynamicProperties'))
                    np.savetxt('dampingRatio.txt', np.array([round(damping,4)]), fmt = '%.3f')

                    # modify cripple wall height
                    cw_H = c
                    os.chdir(os.path.join(current_model_dir, 'Geometry'))
                    storyHeights = np.genfromtxt('storyHeights.txt')
                    storyHeights[0] = cw_H*12
                    np.savetxt('storyHeights.txt', storyHeights, fmt = '%.3f')

                    os.chdir(os.path.join(current_model_dir, 'StructuralProperties/XWoodPanels'))
                    xheight = np.genfromtxt('height.txt')
                    xheight[0][np.where(xheight[0] == 24)] = cw_H*12
                    np.savetxt('height.txt', xheight, fmt = '%.1f', delimiter='\t')

                    os.chdir(os.path.join(current_model_dir, 'StructuralProperties/ZWoodPanels'))
                    zheight = np.genfromtxt('height.txt')
                    zheight[0][np.where(zheight[0] == 24)] = cw_H*12
                    np.savetxt('height.txt', zheight, fmt = '%.1f', delimiter='\t')

                    # modify seismic weight 
                    weight_multiplier = s
                    cw_weight = [i*cw_H/2*CRIPPLE_WALL_WEIGHT/1000 for i in CRIPPLE_WALL_TRIB_LENGTH]
                    w = WEIGHT[1].copy()
                    w[0] = np.add(w[0], cw_weight)
                    w_adjusted = np.multiply(weight_multiplier, w)
                    story_weight = np.sum(w_adjusted, axis = 1)
                    os.chdir(os.path.join(current_model_dir, 'Loads'))
                    np.savetxt('leaningcolumnLoads.txt', w_adjusted, fmt='%.3f', delimiter = '\t')
                    np.savetxt('floorWeights.txt', story_weight, fmt='%.3f', delimiter = '\n')



                    # modify material properties 
                    os.chdir(os.path.join(current_model_dir, 'StructuralProperties/Pinching4Materials'))

                    for p in range(1,5):
                        disp_file = np.genfromtxt('d%i.txt'%p)
                        disp_file[CRIPPLE_WALL_MAT] *= dis
                        disp_file[CRIPPLE_WALL_MAT-1] *= dis

                        disp_file[CRIPPLE_RETRO_MAT] *= dis
                        disp_file[CRIPPLE_RETRO_MAT-1] *= dis

                        disp_file[SUPER_EXT_MAT] *= dis
                        disp_file[SUPER_EXT_MAT-1] *= dis

                        disp_file[SUPER_INT_MAT] *= dis
                        disp_file[SUPER_INT_MAT-1] *= dis

                        np.savetxt('d%i.txt'%p, disp_file, fmt = '%.4f', delimiter = '\n')

                        force_file = np.genfromtxt('f%i.txt'%p)
                        force_file[CRIPPLE_WALL_MAT] *= f
                        force_file[CRIPPLE_WALL_MAT-1] *= f

                        force_file[CRIPPLE_RETRO_MAT] *= f
                        force_file[CRIPPLE_RETRO_MAT-1] *= f

                        force_file[SUPER_EXT_MAT] *= f
                        force_file[SUPER_EXT_MAT-1] *= f

                        force_file[SUPER_INT_MAT] *= f
                        force_file[SUPER_INT_MAT-1] *= f

                        np.savetxt('f%i.txt'%p, force_file, fmt = '%.4f', delimiter = '\n')

                    i += 1


In [3]:
os.chdir(r'C:\Users\User\Desktop\ExperimentDesign\OS Models')
design_mat.to_csv('Design_Mat.csv')

In [8]:
b_list = ['1L-S2-G2-2C-S2R2-SDSx-'+str(i) for i in range(1,244)]
HOME_DIR = r'C:\Users\User\Desktop\ExperimentDesign\OS Models'
# Utility function directory 
UTIL_DIR = os.path.join(HOME_DIR, 'ModelAssembling')
os.chdir(UTIL_DIR)
with open('BuildingName.txt', 'w') as f:
    for b in b_list:

        f.write(b+'\n')
f.close()
# BuildingList = BuildingList.split('\n')
